In [2]:
import torch

## vectorizing encode

In [78]:
i = torch.tensor([0, 0, 1])

In [77]:
wh_conf = torch.rand(3, 4)

In [103]:
boxes = torch.cat([i.unsqueeze(1),wh_conf], dim=1)

In [104]:
boxes

tensor([[0.0000, 0.9579, 0.3726, 0.7087, 0.7728],
        [0.0000, 0.2611, 0.2287, 0.2003, 0.9335],
        [1.0000, 0.3852, 0.4650, 0.8816, 0.4901]])

In [133]:
out = encode_labelsv2(boxes, 3, 2, 2)

In [136]:
out2 = encode_labels(boxes, 3, 2, 2)

0 1 0
0 0 0
0 0 1


In [137]:
out2

tensor([[[-0.1668,  0.6598,  0.9389,  0.7000,  1.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  1.0000,  1.0000],
         [ 0.8105, -0.0414,  0.8419,  0.8791,  1.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  1.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  

In [138]:
out

tensor([[[-0.1668,  0.6598,  0.9389,  0.7000,  1.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  1.0000],
         [ 0.8105, -0.0414,  0.8419,  0.8791,  1.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  1.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  

In [112]:
class_labels = boxes[:, 0].long()
cx, cy = boxes[:, 1], boxes[:, 2]
w, h = boxes[:, 3], boxes[:, 4]

In [115]:
x, y = cx - w * 0.5, cy - h * 0.5

# Calculate the cell indices
i, j = (3 * y).long(), (3 * x).long()

# Calculate the cell-relative coordinates
x_cell, y_cell = 3 * x - j, 3 * y - i

In [116]:
stacked = torch.stack((x_cell, y_cell, torch.sqrt(w), torch.sqrt(h), torch.ones_like(x_cell)), dim=-1)

In [119]:
stacked.shape, class_labels.shape

(torch.Size([3, 5]), torch.Size([3]))

In [121]:
class_probs = torch.zeros((boxes.shape[0], 2))


In [122]:
class_probs.shape

torch.Size([3, 2])

In [123]:
class_probs.scatter_(1, class_labels.unsqueeze(1), 1)


tensor([[1., 0.],
        [1., 0.],
        [0., 1.]])

In [129]:
torch.cat((stacked, class_probs), dim=-1).size()

torch.Size([3, 7])

In [132]:


def encode_labelsv2(labels: torch.Tensor, S, B, C) -> torch.Tensor:
    """SxS grid according to yolo algorithm
    labels: tensor of shape [N, 5]
    """
    encoded_labels = torch.zeros((S, S, B * 5 + C))

    # shapes: torch.Size([N]))
    class_labels = labels[:, 0].int()
    cx, cy = labels[:, 1], labels[:, 2]
    w, h = labels[:, 3], labels[:, 4]
    x, y = cx - w * 0.5, cy - h * 0.5

    # Calculate the cell indices and cell-rel coords
    i, j = (S * y).int(), (S * x).int()
    x_cell, y_cell = S * x - j, S * y - i

    # stacked shape: torch.Size([N, 5])
    boxes = torch.stack((x_cell, y_cell, torch.sqrt(w), torch.sqrt(h), torch.ones_like(x_cell)), dim=-1)

    # Create a tensor to store the class probabilities (OHE)
    # scatter_ puts 1 on indices defined by class_labels.unsqueeze(1)
    # OHE shape: torch.Size([N, C])
    class_probs = torch.zeros((labels.shape[0], C))
    class_probs.scatter_(1, class_labels.unsqueeze(1), 1)

    # No matter what B is, we make only the first box responsible
    encoded_labels[i, j, :5] = boxes
    encoded_labels[i, j, -C:] = class_probs

    return encoded_labels


def encode_labels(labels: torch.Tensor, S, B, C) -> torch.Tensor:
    """SxS grid according to yolo algorithm
    labels: tensor of shape [N, 5]
    """
    encoded_labels = torch.zeros((S, S, B * 5 + C))
    for label in labels:
        class_label, cx, cy, w, h = label
        x, y = cx - w * 0.5, cy - h * 0.5
        class_label = int(class_label)
        
        i, j = int(S * y), int(S * x)
        x_cell, y_cell = S * x - j, S * y - i

        # set the first box with true labels
        encoded_labels[i, j, 0:4] = torch.tensor(
            [x_cell, y_cell, torch.sqrt(w), torch.sqrt(h)]
        )
        # Object conf set to 1 for simplificty
        # replace with IoU during loss computation
        encoded_labels[i, j, 4] = 1
        print(i, j, class_label)
        # Encode the class probabilities (OHE)
        encoded_labels[i, j, -C + class_label] = 1

    return encoded_labels

## vectorizing decode

In [3]:
# Example dimensions
B = 3
C = 2
x = torch.randn(10, B*5 + C)

In [4]:
x[0]

tensor([ 2.9616,  0.1236,  2.7516,  1.6306,  0.6964, -0.7375, -0.2009,  0.4031,
         1.7019,  1.4073, -0.2385, -0.0675,  1.5605,  0.3064,  1.6552,  1.0706,
        -0.5794])

In [7]:
boxes = x[..., :B*5]

In [14]:
confs = x[..., -C:]

In [27]:
confs

tensor([[ 1.0706, -0.5794],
        [-1.3386, -1.6600],
        [ 0.2133,  2.7711],
        [-1.3419,  1.0153],
        [-0.5589,  1.9114],
        [ 0.2992, -0.6253],
        [-0.7575,  1.1813],
        [ 1.2572, -2.1036],
        [-0.3777, -1.4239],
        [-0.1400, -0.1314]])

In [30]:
confs.repeat_interleave(B, 0).shape

torch.Size([30, 2])

In [19]:
x = torch.tensor([1, 2, 3])

In [18]:
confs.repeat?

Docstring:
repeat(*sizes) -> Tensor

Repeats this tensor along the specified dimensions.

Unlike :meth:`~Tensor.expand`, this function copies the tensor's data.

.. warning::

    :meth:`~Tensor.repeat` behaves differently from
    `numpy.repeat <https://docs.scipy.org/doc/numpy/reference/generated/numpy.repeat.html>`_,
    but is more similar to
    `numpy.tile <https://docs.scipy.org/doc/numpy/reference/generated/numpy.tile.html>`_.
    For the operator similar to `numpy.repeat`, see :func:`torch.repeat_interleave`.

Args:
    sizes (torch.Size or int...): The number of times to repeat this tensor along each
        dimension

Example::

    >>> x = torch.tensor([1, 2, 3])
    >>> x.repeat(4, 2)
    tensor([[ 1,  2,  3,  1,  2,  3],
            [ 1,  2,  3,  1,  2,  3],
            [ 1,  2,  3,  1,  2,  3],
            [ 1,  2,  3,  1,  2,  3]])
    >>> x.repeat(4, 2, 1).size()
    torch.Size([4, 2, 3])
Type:      builtin_function_or_method

In [8]:
boxes[0]

tensor([ 2.9616,  0.1236,  2.7516,  1.6306,  0.6964, -0.7375, -0.2009,  0.4031,
         1.7019,  1.4073, -0.2385, -0.0675,  1.5605,  0.3064,  1.6552])

In [9]:
boxes.reshape(10*B, -1)[0]

tensor([2.9616, 0.1236, 2.7516, 1.6306, 0.6964])

In [12]:
boxes.reshape(10*B, -1)[2]

tensor([-0.2385, -0.0675,  1.5605,  0.3064,  1.6552])

In [13]:
boxes.reshape(10*B, -1).shape

torch.Size([30, 5])

In [ ]:
confs.repeat_interleave(B, 0)

In [31]:
class_probs = confs.repeat_interleave(B, dim=0)

In [33]:
boxes = boxes.reshape(10*B, -1)

In [34]:
boxes.shape, class_probs.shape

(torch.Size([30, 5]), torch.Size([30, 2]))

In [ ]:
merged = torch.cat([boxes, class_probs

In [70]:
class_probs, class_idx = torch.max(confs,dim=-1, keepdim=True)

In [71]:
class_idx

tensor([[0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1]])

In [72]:
class_probs

tensor([[ 1.0706],
        [-1.3386],
        [ 2.7711],
        [ 1.0153],
        [ 1.9114],
        [ 0.2992],
        [ 1.1813],
        [ 1.2572],
        [-0.3777],
        [-0.1314]])

In [74]:
class_idx_rep = class_idx.repeat_interleave(B, dim=0)

In [76]:
class_idx_rep.shape

torch.Size([30, 1])

In [79]:
torch.cat([boxes, class_idx_rep], dim=-1)

tensor([[ 2.9616,  0.1236,  2.7516,  1.6306,  0.6964,  0.0000],
        [-0.7375, -0.2009,  0.4031,  1.7019,  1.4073,  0.0000],
        [-0.2385, -0.0675,  1.5605,  0.3064,  1.6552,  0.0000],
        [-0.7461,  1.6173,  0.2564,  0.4192, -0.0912,  0.0000],
        [ 1.3360, -0.3270,  0.7847,  0.5295,  0.4127,  0.0000],
        [ 0.5834, -1.7876,  2.3975,  0.8374, -0.0284,  0.0000],
        [-0.2077,  1.7653,  1.1105,  0.3915, -0.5637,  1.0000],
        [ 0.8680,  0.9730, -1.3465, -1.0940,  0.1654,  1.0000],
        [-0.0132, -0.2464, -0.0919, -0.7040,  1.6186,  1.0000],
        [ 1.8175,  1.7617,  2.6011,  0.5624, -0.3884,  1.0000],
        [-1.0491, -0.0615, -2.2598,  1.7637, -0.5039,  1.0000],
        [ 0.4838, -0.1474,  0.7951, -0.2404,  0.3210,  1.0000],
        [ 0.4940,  0.9573,  0.5390,  0.6672, -1.2127,  1.0000],
        [ 2.0386, -1.5385, -0.4422,  0.5366, -0.0364,  1.0000],
        [ 1.6713,  0.6843, -0.4253, -1.4757, -0.8788,  1.0000],
        [ 0.6084, -0.0284, -0.0777, -0.1

In [86]:
i_ = torch.range(0, 9)

/tmp/ipykernel_587/4257354964.py:1: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  i_ = torch.range(0, 9)


In [94]:
i_.repeat(10)

tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9., 0., 1., 2., 3., 4., 5., 6., 7.,
        8., 9., 0., 1., 2., 3., 4., 5., 6., 7., 8., 9., 0., 1., 2., 3., 4., 5.,
        6., 7., 8., 9., 0., 1., 2., 3., 4., 5., 6., 7., 8., 9., 0., 1., 2., 3.,
        4., 5., 6., 7., 8., 9., 0., 1., 2., 3., 4., 5., 6., 7., 8., 9., 0., 1.,
        2., 3., 4., 5., 6., 7., 8., 9., 0., 1., 2., 3., 4., 5., 6., 7., 8., 9.,
        0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [98]:
confs

tensor([[ 1.0706, -0.5794],
        [-1.3386, -1.6600],
        [ 0.2133,  2.7711],
        [-1.3419,  1.0153],
        [-0.5589,  1.9114],
        [ 0.2992, -0.6253],
        [-0.7575,  1.1813],
        [ 1.2572, -2.1036],
        [-0.3777, -1.4239],
        [-0.1400, -0.1314]])

In [101]:
confs[..., 1]

tensor([-0.5794, -1.6600,  2.7711,  1.0153,  1.9114, -0.6253,  1.1813, -2.1036,
        -1.4239, -0.1314])

In [105]:
confs[..., 1].pow_(2)

TypeError: TensorBase.square_() takes no arguments (1 given)

In [93]:
i_.repeat_interleave(10)

torch.Size([100, 1])

In [248]:
def decode_labelsv0(
    encoded_labels,
    S,
    B,
    C,
    conf_th=0.1,
    nms=False,
    iou_th=0.5,
):
    labels = []

    for i in range(S):
        for j in range(S):
            cell_output = encoded_labels[i, j]
            bbox = cell_output[:4]
            conf = cell_output[4]
            class_probs = cell_output[5:]

            x_cell, y_cell, w_cell, h_cell = bbox
            x = (j + x_cell) / S
            y = (i + y_cell) / S
            w = w_cell**2
            h = h_cell**2
            cx, cy = x + w * 0.5, y + h * 0.5

            class_idx = torch.argmax(class_probs)
            class_prob = class_probs[class_idx] * conf

            # print(class_idx, class_prob, conf)
            # return

            if class_prob > conf_th:
                box = torch.tensor(
                    [
                        class_idx.item(),
                        cx,
                        cy,
                        w,
                        h,
                        conf,
                        class_prob.item(),
                    ]
                )
                labels.append(box.unsqueeze(0))

    results = torch.empty(size=(0, 7), dtype=torch.float32)
    if labels:
        results = torch.cat(labels, dim=0)
        if nms:
            results = apply_nms(results, iou_th)

    return results




In [278]:
def decode_vec(encoded_labels,
               S=3, B=2, C=2, prob_th=0.1,
               nms=True, iou_th=0.5):
    """
    ip shape: [S, S, B*5 + C]
    """
    if encoded_labels.numel() 
    # view shape: [S*S, B*5 + C]
    
    encoded_labels = encoded_labels.view(S*S, -1)
    # extracted cell boxes shape: [S*S*B, 5]
    boxes = encoded_labels[:, :B*5].reshape(S*S*B, -1)
    # decoding OHE class probs
    class_probs, class_idx = torch.max(encoded_labels[:, -C:], dim=-1, keepdim=True)
    class_probs = class_probs.repeat_interleave(B, dim=0)
    class_idx = class_idx.repeat_interleave(B, dim=0)
    # shape: [S*S*B, 7]
    boxes_and_probs = torch.cat([class_idx, boxes, class_probs], dim=-1)

    # decoding boxes to cxcywh format
    i = torch.arange(S).repeat_interleave(S*B)
    j = torch.arange(S).repeat(S).repeat_interleave(B)
    x = ((boxes_and_probs[:, 1] + i) / S) + boxes_and_probs[:, 3]*0.5
    
    # w and h were square rooted during encoding.
    boxes_and_probs[:, 3:5].pow_(2)
    boxes_and_probs[:, 1] = ((boxes_and_probs[:, 1] + j) / S) + boxes_and_probs[:, 3]*0.5
    boxes_and_probs[:, 2] = ((boxes_and_probs[:, 2] + i) / S) + boxes_and_probs[:, 4]*0.5
    # p(class) = p(class | obj) * p(obj)
    boxes_and_probs[:, -1] *= boxes_and_probs[:, 5]
    prob_mask = boxes_and_probs[:, -1] > prob_th
    valid_boxes_and_probs = boxes_and_probs[prob_mask]

    print(valid_boxes_and_probs.shape)
    return valid_boxes_and_probs
    

In [250]:
S, B, C = 3, 1, 4

In [269]:
ip = torch.rand(S, S, B*5 + C)

In [270]:
decoded0 = decode_labelsv0(ip, S, B, C, conf_th=0.0)

In [281]:
decoded = decode_vec(torch.tensor([]), S, B, C, prob_th=0.0)

IndexError: max(): Expected reduction dim 1 to have non-zero size.

In [272]:
decoded0[1]

tensor([2.0000, 0.6329, 0.2927, 0.0034, 0.3312, 0.2901, 0.2232])

In [273]:
decoded[1]

tensor([2.0000, 0.6329, 0.2927, 0.0034, 0.3312, 0.2901, 0.2232])

In [274]:
decoded == decoded0

tensor([[True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True]])

In [283]:
torch.arange(

device(type='cpu')

## custom loss

In [292]:
confs

tensor([[ 1.0706,  0.0000],
        [-1.3386,  1.0000],
        [ 0.2133,  2.0000],
        [-1.3419,  3.0000],
        [-0.5589,  4.0000],
        [ 0.2992,  5.0000],
        [-0.7575,  6.0000],
        [ 1.2572,  7.0000],
        [-0.3777,  8.0000],
        [-0.1400,  9.0000]])

In [293]:
confs.repeat(1, 2)

tensor([[ 1.0706,  0.0000,  1.0706,  0.0000],
        [-1.3386,  1.0000, -1.3386,  1.0000],
        [ 0.2133,  2.0000,  0.2133,  2.0000],
        [-1.3419,  3.0000, -1.3419,  3.0000],
        [-0.5589,  4.0000, -0.5589,  4.0000],
        [ 0.2992,  5.0000,  0.2992,  5.0000],
        [-0.7575,  6.0000, -0.7575,  6.0000],
        [ 1.2572,  7.0000,  1.2572,  7.0000],
        [-0.3777,  8.0000, -0.3777,  8.0000],
        [-0.1400,  9.0000, -0.1400,  9.0000]])

In [344]:
N, S, B, C = 2, 3, 2, 4

In [345]:
preds = torch.rand(N, S, S, B*5 + C)

In [346]:
targets = torch.rand(N, S, S, B*5 + C)

In [347]:
preds.shape

torch.Size([2, 3, 3, 14])

In [289]:
mask_obj_exists = targets[..., 4] > 0

In [291]:
mask_obj_exists.shape

torch.Size([2, 3, 3])

In [294]:
confs

tensor([[ 1.0706,  0.0000],
        [-1.3386,  1.0000],
        [ 0.2133,  2.0000],
        [-1.3419,  3.0000],
        [-0.5589,  4.0000],
        [ 0.2992,  5.0000],
        [-0.7575,  6.0000],
        [ 1.2572,  7.0000],
        [-0.3777,  8.0000],
        [-0.1400,  9.0000]])

In [297]:
confs.max(dim=1, keepdim=True).values

tensor([[1.0706],
        [1.0000],
        [2.0000],
        [3.0000],
        [4.0000],
        [5.0000],
        [6.0000],
        [7.0000],
        [8.0000],
        [9.0000]])

In [298]:
x = torch.randn(N, S*S, B, 7)

In [299]:
x.shape

torch.Size([2, 9, 2, 7])

In [300]:
mask = x[..., 5] > 0

In [301]:
mask.shape

torch.Size([2, 9, 2])

In [302]:
mask2 = torch.ones(2, 9)

In [303]:
mask2.shape

torch.Size([2, 9])

In [307]:
x[mask.int()].shape

torch.Size([2, 9, 2, 9, 2, 7])

In [308]:
x.shape

torch.Size([2, 9, 2, 7])

In [589]:
def calc_iou(pred_boxes, target_boxes):
    pred_x1 = pred_boxes[..., 0] - pred_boxes[..., 2] / 2
    pred_y1 = pred_boxes[..., 1] - pred_boxes[..., 3] / 2
    pred_x2 = pred_boxes[..., 0] + pred_boxes[..., 2] / 2
    pred_y2 = pred_boxes[..., 1] + pred_boxes[..., 3] / 2

    target_x1 = target_boxes[..., 0] - target_boxes[..., 2] / 2
    target_y1 = target_boxes[..., 1] - target_boxes[..., 3] / 2
    target_x2 = target_boxes[..., 0] + target_boxes[..., 2] / 2
    target_y2 = target_boxes[..., 1] + target_boxes[..., 3] / 2

    inter_x1 = torch.max(pred_x1, target_x1)
    inter_y1 = torch.max(pred_y1, target_y1)
    inter_x2 = torch.min(pred_x2, target_x2)
    inter_y2 = torch.min(pred_y2, target_y2)

    inter_area = torch.clamp(inter_x2 - inter_x1, min=0) * torch.clamp(inter_y2 - inter_y1, min=0)

    pred_area = (pred_x2 - pred_x1) * (pred_y2 - pred_y1)
    target_area = (target_x2 - target_x1) * (target_y2 - target_y1)

    union_area = pred_area + target_area - inter_area

    iou = inter_area / union_area
    return iou

In [608]:
def loss(preds, targets, S, B, C):
    """
    preds, targets: [N, S, S, B*5 + C]
    """
    mse = torch.nn.MSELoss(reduction="sum")
    # shape: [N, S*S*B, 7] in (class_idx, cx, cy, w, h, conf, cond_cls_prob)
    N = len(targets)
    decoded_preds = torch.stack([
        decode_labels(x, S, B, C, prob_th=-1, nms=False)
        for x in preds])
    decoded_targets = torch.stack([
        decode_labels(x, S, B, C, prob_th=-1, nms=False)
        for x in targets])
    last_dim_sz = decoded_targets.shape[-1]
    
    # obj mask shape: [N, S*S*B, 7]
    mask_obj_exists = decoded_targets[..., 5] > 0
    mask_obj_exists = mask_obj_exists.unsqueeze(-1).expand_as(decoded_targets)
    # print(mask_obj_exists)

    # shape: [N, K, 4] K is a multiple of B since we had 3 times
    preds_cls_filtered = preds.view(N, S*S, -1)[..., -C]
    # preds_cls_filtered[
    
    preds_filtered = decoded_preds[mask_obj_exists].view(N, -1, last_dim_sz)
    targets_filtered = decoded_targets[mask_obj_exists].view(N, -1, last_dim_sz)
    preds_unfiltered = decoded_preds[~mask_obj_exists].view(N, -1, last_dim_sz)
    targets_unfiltered = decoded_targets[~mask_obj_exists].view(N, -1, last_dim_sz)

    # shape: [N*K]
    ious = calc_iou(preds_filtered[..., 1:5].reshape(-1, 4), 
                    targets_filtered[..., 1:5].reshape(-1, 4))
    
    # shape: [N, K/B, B]
    ious = ious.reshape(N, -1, B)

    # shape: [N, K, 4]
    _, best_iou_idx = torch.max(ious, dim=-1, keepdim=False)
    best_iou_mask = torch.zeros_like(ious)
    best_iou_mask = best_iou_mask.scatter(-1, 
                                          best_iou_idx.unsqueeze(-1), 
                                          1,
                                         ).reshape(N, -1)
    best_iou_mask = best_iou_mask.unsqueeze(-1).bool()

    # coord loss
    pred_boxes = preds_filtered[..., 1:5]
    target_boxes = targets_filtered[..., 1:5]
    resp_pred_boxes = pred_boxes[best_iou_mask.expand_as(pred_boxes)].view(-1, 4)
    resp_target_boxes = target_boxes[best_iou_mask.expand_as(target_boxes)].view(-1, 4)
    
    coord_loss = mse(resp_pred_boxes, resp_target_boxes)

    # obj loss
    pred_confs = preds_filtered[..., 5:6]
    target_confs = targets_filtered[..., 5:6]
    resp_pred_confs = pred_confs[best_iou_mask.expand_as(pred_confs)].view(-1, 1)
    resp_target_confs = target_confs[best_iou_mask.expand_as(target_confs)].view(-1, 1)

    obj_loss = mse(resp_pred_confs, resp_target_confs)

    # noobj loss
    noobj_pred_confs = preds_unfiltered[..., 5:6]
    noobj_target_confs = targets_unfiltered[..., 5:6]
    nonresp_pred_confs = pred_confs[~best_iou_mask.expand_as(pred_confs)].view(N, -1, 1)
    nonresp_target_confs = torch.zeros_like(nonresp_pred_confs)
    noobj_pred_confs = torch.cat([noobj_pred_confs, nonresp_pred_confs], dim=1)
    noobj_target_confs = torch.cat([noobj_target_confs, nonresp_target_confs], dim=1)

    noobj_loss = mse(noobj_pred_confs, noobj_target_confs)

    # class loss
    mask_cls = mask_obj_exists.reshape(N, S, S, -1)[..., 0:1].expand_as(preds)
    pred_cls_probs = preds[mask_cls][..., -C:]
    target_cls_probs = targets[mask_cls][..., -C:]
    
    cls_loss = mse(pred_cls_probs, target_cls_probs)
    
    print(obj_loss, coord_loss, noobj_loss, cls_loss)

In [421]:
import yolov1
import pprint
from yolov1.config import parse_config
from yolov1.data.dataset import YOLODataset
from yolov1.utils.general import decode_labels

In [392]:
def load_config(show=False):
    config_path = "../../yolov1/src/yolov1/configs/default.yaml"
    config = parse_config(config_file=config_path)
    if show:
        pprint.pp(config.dict())
    return config

In [403]:
config = load_config()

In [404]:
S, B, C = config.model.S, config.model.B, config.model.nc
S, B, C

(3, 2, 5)

In [405]:
val_ds = YOLODataset(config, mode="valid", encode=True)

2024-05-16T18:40:42.708574Z [info     ] Loaded 250 samples             func_name=get_data lineno=88
2024-05-16T18:40:42.709084Z [info     ] Class weights: [0.4, 0.24, 0.12, 0.36, 0.32] func_name=_set_class_weights lineno=56


In [406]:
first = val_ds[0]

In [410]:
encoded_targets = first[1].unsqueeze(0)

In [412]:
encoded_targets.shape

torch.Size([1, 3, 3, 15])

In [411]:
encoded_targets

tensor([[[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.6677, 0.9285, 0.3968, 0.4891, 1.0000, 0.6677, 0.9285, 0.3968,
           0.4891, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000],
          [0.7224, 0.8041, 0.1297, 0.1387, 1.0000, 0.7224, 0.8041, 0.1297,
           0.1387, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000]],

         [[0.0018, 0.0349, 0.5894, 0.5119, 1.0000, 0.0018, 0.0349, 0.5894,
           0.5119, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000],
          [0.8624, 0.4008, 0.6140, 0.7286, 1.0000, 0.8624, 0.4008, 0.6140,
           0.7286, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.00

In [595]:
preds = torch.rand(2, S, S, B*5 + C)

In [596]:
preds.shape

torch.Size([2, 3, 3, 15])

In [597]:
preds

tensor([[[[0.9260, 0.1062, 0.4728, 0.4427, 0.4072, 0.0135, 0.3873, 0.8887,
           0.6450, 0.9028, 0.5523, 0.4007, 0.1212, 0.6134, 0.9612],
          [0.2879, 0.8812, 0.0099, 0.2160, 0.5365, 0.8191, 0.0318, 0.6130,
           0.0723, 0.3746, 0.3100, 0.1695, 0.0492, 0.3387, 0.6989],
          [0.8175, 0.5271, 0.2679, 0.8604, 0.5091, 0.6466, 0.9669, 0.0662,
           0.7394, 0.3783, 0.2177, 0.0232, 0.4700, 0.0303, 0.3568]],

         [[0.3673, 0.8453, 0.3735, 0.0249, 0.1917, 0.8579, 0.3690, 0.4156,
           0.2589, 0.9686, 0.5306, 0.5124, 0.4264, 0.1917, 0.4729],
          [0.5257, 0.1256, 0.0782, 0.9726, 0.4917, 0.3759, 0.5928, 0.9342,
           0.4513, 0.9643, 0.3129, 0.0758, 0.7433, 0.5578, 0.5049],
          [0.0554, 0.2177, 0.1641, 0.6000, 0.8540, 0.7621, 0.3091, 0.2649,
           0.0534, 0.6133, 0.3350, 0.3696, 0.8832, 0.7284, 0.3935]],

         [[0.6074, 0.8624, 0.6181, 0.0153, 0.0486, 0.8517, 0.1839, 0.4805,
           0.3152, 0.7925, 0.1649, 0.9371, 0.7676, 0.7113, 0.07

In [610]:
loss(preds, torch.cat([encoded_targets,
                        encoded_targets]), S, B, C)

torch.Size([2, 3, 3, 15]) torch.Size([2, 3, 3, 15])
tensor(0.7881) tensor(1.9949) tensor(10.2811) tensor(1.3809)


In [609]:
loss(preds, encoded_targets, S, B, C)

torch.Size([2, 3, 3, 15]) torch.Size([1, 3, 3, 15])


IndexError: The shape of the mask [1, 18, 7] at index 0 does not match the shape of the indexed tensor [2, 18, 7] at index 0

In [594]:
xx = torch.tensor([0.0009, 0.0696, 0.0000, 0.0000, 0.0540, 0.0791, 0.0000, 0.1470])

In [479]:
xx.shape

torch.Size([8])

In [480]:
xx

tensor([0.0009, 0.0696, 0.0000, 0.0000, 0.0540, 0.0791, 0.0000, 0.1470])

In [469]:
yy = torch.tensor([[1, 0, 1, 1]])

In [511]:
iou_mask = torch.zeros_like(xx).reshape(4, 2)

In [512]:
iou_mask

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])

In [514]:
iou_mask.scatter(1, yy.reshape(4, 1), 1).reshape(xx.shape)

tensor([0., 1., 1., 0., 0., 1., 0., 1.])

In [566]:
preds

tensor([[[[0.1014, 0.3509, 0.7096, 0.3590, 0.5059, 0.8760, 0.4589, 0.4028,
           0.1521, 0.9241, 0.5535, 0.6976, 0.5283, 0.1457, 0.7197],
          [0.4522, 0.4449, 0.2687, 0.8649, 0.4985, 0.6969, 0.3481, 0.8812,
           0.8068, 0.5763, 0.6155, 0.0775, 0.8457, 0.9575, 0.4400],
          [0.0173, 0.3034, 0.9779, 0.3435, 0.9939, 0.2570, 0.2705, 0.9680,
           0.1560, 0.5211, 0.2091, 0.1780, 0.6047, 0.8249, 0.7159]],

         [[0.6490, 0.6258, 0.4817, 0.5723, 0.9932, 0.7217, 0.4252, 0.3292,
           0.9708, 0.4000, 0.2672, 0.8188, 0.9201, 0.4729, 0.3722],
          [0.5451, 0.7504, 0.2668, 0.9280, 0.9178, 0.2815, 0.9798, 0.9263,
           0.9619, 0.4463, 0.4913, 0.0248, 0.2675, 0.8094, 0.1555],
          [0.8827, 0.4363, 0.6274, 0.2234, 0.4448, 0.0914, 0.3879, 0.4188,
           0.7209, 0.3812, 0.4332, 0.3124, 0.2081, 0.4082, 0.1410]],

         [[0.0971, 0.9668, 0.5396, 0.1257, 0.2578, 0.9445, 0.7895, 0.7754,
           0.4941, 0.9910, 0.1739, 0.5148, 0.6078, 0.1965, 0.13

In [567]:
preds.view(1, S*S*B, -1)

RuntimeError: shape '[1, 18, -1]' is invalid for input of size 135

In [568]:
preds.shape

torch.Size([1, 3, 3, 15])

In [611]:
96/32

3.0